In [ ]:
!pip install --upgrade pixiedust
!pip install --upgrade bokeh

# pixiedust.installPackage("cloudant-labs:spark-cloudant:2.0.0-s_2.11")

In [ ]:
import pixiedust
import bokeh

In [ ]:
# The code was removed by Watson Studio for sharing.

In [ ]:
sparkSession = SQLContext.getOrCreate(sc).sparkSession

cloudantdata=sparkSession.read.format("com.cloudant.spark")\
  .option("cloudant.host",credentials_1['host'])\
  .option("cloudant.username", credentials_1['username'])\
  .option("cloudant.password", credentials_1['password'])\
  .option("jsonstore.rdd.partitions", "1")\
  .option("inferSchema", "True")\
  .load("zzshake")

cloudantdata.createOrReplaceTempView("dataframe")
cloudantdata.printSchema

In [ ]:
sqlenergy = spark.sql("select cast (sum(sqrt((AX * AX) + (AY * AY) + (AZ * AZ))) as integer) as EN, ID from dataframe group by ID order by EN desc")
sqlenergy.show()

In [ ]:
display(sqlenergy)

In [ ]:
modeldata = spark.sql("select TS, ID, AX, AY, AZ, OA, OB, OG, sqrt((AX * AX) + (AY * AY) + (AZ * AZ)) as EN from dataframe")
display(modeldata)

In [ ]:
display(modeldata)

In [ ]:
display(modeldata)

In [ ]:
modeldata

In [ ]:
modeldata.show()

In [ ]:
modeldata.printSchema()

In [ ]:
modeldata.describe().show()

In [ ]:
modeldata.count()

In [ ]:
split_data = modeldata.randomSplit([0.5, 0.5], 0)
train_data = split_data[0]
test_data  = split_data[1]

print "Number of training records : " + str(train_data.count())
print "Number of testing records  : " + str(test_data.count())

In [ ]:
from pyspark.ml.feature import StringIndexer, IndexToString, VectorAssembler

from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import GBTClassifier

from pyspark.ml.evaluation import MulticlassClassificationEvaluator

from pyspark.ml import Pipeline, Model

In [ ]:
stringIndexer_label = StringIndexer(inputCol = "ID", outputCol = "label").fit(modeldata)
stringIndexer_label.labels

In [ ]:
vectorAssembler_features = VectorAssembler(inputCols=["AX", "AY", "AZ", "OA", "OB", "OG"], outputCol = "features")

In [ ]:
labelConverter = IndexToString(inputCol = "prediction", outputCol = "predlabel", labels = stringIndexer_label.labels)

In [ ]:
cf = RandomForestClassifier(labelCol = "label", featuresCol = "features")

# cf = GBTClassifier(labelCol="label", featuresCol="features")

In [ ]:
pipeline_cf = Pipeline(stages = [stringIndexer_label, vectorAssembler_features, cf, labelConverter])

In [ ]:
model_cf = pipeline_cf.fit(train_data)

In [ ]:
predictions = model_cf.transform(test_data)
evaluator = MulticlassClassificationEvaluator(labelCol = "label", predictionCol = "prediction", metricName = "accuracy")
accuracy = evaluator.evaluate(predictions)

print("Accuracy = %g" % accuracy)

In [ ]:
predictions.show()

In [ ]:
display(predictions)